In [ ]:
!pip install datasets

In [ ]:
import datasets

snli = datasets.load_dataset('snli', split='train')

snli


mnli = datasets.load_dataset('glue', 'mnli', split='train')

mnli = mnli.remove_columns(['idx'])


snli = snli.cast(mnli.features)

dataset = datasets.concatenate_datasets([snli, mnli])

del snli, mnli

In [ ]:


print(f"before: {len(dataset)} rows")
dataset = dataset.filter(
    lambda x: True if x['label'] == 0 else False
)
print(f"after: {len(dataset)} rows")



In [ ]:
#here we install the necessary packages. feel free to comment this part if you already have it installed
!pip install sentence-transformers

from sentence_transformers import InputExample
from tqdm.auto import tqdm  # so we see progress bar

train_samples = []
for row in tqdm(dataset):
    train_samples.append(InputExample(
        texts=[row['premise'], row['hypothesis']]
    ))



In [ ]:
from sentence_transformers import datasets

batch_size = 32

loader = datasets.NoDuplicatesDataLoader(
    train_samples, batch_size=batch_size)

In [ ]:
from sentence_transformers import models, SentenceTransformer

bert = models.Transformer('bert-base-uncased')
pooler = models.Pooling(
    bert.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True
)

model = SentenceTransformer(modules=[bert, pooler])

model

In [ ]:
from sentence_transformers import losses

loss = losses.MultipleNegativesRankingLoss(model)

In [ ]:


epochs = 1
warmup_steps = int(len(loader) * epochs * 0.1)

model.fit(
    train_objectives=[(loader, loss)],
    epochs=epochs,
    warmup_steps=warmup_steps,
    output_path='./sbert_test_mnr2',
    show_progress_bar=False
) 


In [ ]:
 sentences = [
    "the fifty mannequin heads floating in the pool kind of freaked them out",
    "she swore she just saw her sushi move",
    "he embraced his new life as an eggplant",
    "my dentist tells me that chewing bricks is very bad for your teeth",
    "the dental specialist recommended an immediate stop to flossing with construction materials",
    "i used to practice weaving with spaghetti three hours a day",
    "the white water rafting trip was suddenly halted by the unexpected brick wall",
    "the person would knit using noodles for a few hours daily",
    "it was always dangerous to drive with him since he insisted the safety cones were a slalom course",
    "the woman thinks she saw her raw fish and rice change position"
]

In [ ]:
embedds = model.encode(sentneces)

In [ ]:
import numpy as np
from sentece_transformers import cos_sim as cs

def simX(model):
  sim = n.zeros((len(sentences), len(sentences)))
  for i in range(len(sentences)):
    sim[i:,i] = cs(embeds[i], embeds[i:])

  return sim

In [ ]:
import matplotlib.py as plt
import seaborn as sns

sim = simX(model)
sns.heatmap(sim, annot= True)